In [ ]:
import numpy as np
import nca
from matplotlib import pyplot as plt


## Define model

In [ ]:
## takes about 2 min
U = 10.
mu = 4.
g = 2.
D = 20.
Gamma = 2 * g**2 / D
beta = 10.
mu_L = -1.
mu_R = 0.5
time_mesh = nca.Mesh(200., int(1e5)).adjoint()

dos = nca.make_gaussian_dos(D)
delta_L_grea, delta_L_less = nca.make_hyb_times(dos, beta, mu_L, Gamma * dos(0.) / dos(mu_L), time_mesh)
delta_R_grea, delta_R_less = nca.make_hyb_times(dos, beta, mu_R, Gamma * dos(0.) / dos(mu_R), time_mesh)

H_loc = np.asarray([0., -mu, -mu, -2*mu + U])

S = nca.SolverSteadyState(2, H_loc, time_mesh, orbital_names=['up', 'dn'])
S.add_bath(0, lambda t: delta_L_grea(t) + delta_R_grea(t), lambda t: delta_L_less(t) + delta_R_less(t))
S.add_bath(1, lambda t: delta_L_grea(t) + delta_R_grea(t), lambda t: delta_L_less(t) + delta_R_less(t))


In [ ]:
times_plot = np.linspace(-1, 1, 300)

plt.plot(times_plot, delta_L_grea(times_plot).real, '-b')
plt.plot(times_plot, delta_L_grea(times_plot).imag, '-r')

plt.show()

plt.plot(times_plot, delta_R_grea(times_plot).real, '-b')
plt.plot(times_plot, delta_R_grea(times_plot).imag, '-r')

plt.show()

## Run computation

In [ ]:
S.greater_loop(max_iter=20, verbose=True)

print("Normalization error:", S.get_normalization_error())
print()

S.lesser_loop(verbose=True)

## Get results: R and S

In [ ]:
R_grea_w = S.get_R_grea_w()
R_less_w = S.get_R_less_w()

S_grea_w = S.get_S_grea_w()
S_less_w = S.get_S_less_w()

In [ ]:
S.state_space.basis

In [ ]:
for i in range(4):
    plt.plot(S.freq_mesh, -R_grea_w[:, i], '-', label=f'state "{S.state_space.basis[i]}"')

plt.xlim(-20, 20)
# plt.semilogy()
plt.legend()

plt.show()

In [ ]:

for i in range(4):
    plt.plot(S.freq_mesh, S_grea_w[:, i].real, '-', label=f'state "{S.state_space.basis[i]}"')
    
plt.xlim(-10, 25)
plt.ylim(-3, 1)
plt.legend()

plt.show()

## Get results: Green function

In [ ]:
freq_mesh, gf = S.get_G_reta_w(0) # orbital 0 is up

In [ ]:
plt.plot(freq_mesh, -gf.imag / np.pi)

plt.xlim(-15, 15)

plt.show()